In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score 

In [ ]:
!pip install scikit-learn=

In [ ]:
df_rt = pd.read_csv("rt_4w_20190709.csv")

In [ ]:
df_sub = pd.read_csv("subcatlog_20190722.csv")

In [66]:
df = pd.read_csv("subcatlog_20190722.csv")
#df["recency"] = np.log(df["recency"]+0.00001)
#df["inview_recency"] = np.log(df["inview_recency"]+0.00001)
# 欠損があるカラムをデフォルト値で埋める
df["region"] = df["region"].fillna(-1).astype(int)

# テーブルをLEFT JOINする(キー: req_id, sponsor_id)
#df=df.merge(df_rt, on=["ssp_uid", "sponsor_id"], how="left")

# カラムを日時として解釈しなおす
df["request_time"] = pd.to_datetime(df["request_time"])

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
df.loc[df["recency"]==0, "recency"]=100000
df.loc[df["inview_recency"]==0, "inview_recency"]=100000

In [82]:
df["recency"] = np.log(df["recency"])
df["inview_recency"] = np.log(df["inview_recency"])

In [83]:
df["recency"].describe()

count    90093.000000
mean         4.821323
std          3.192922
min          0.000000
25%          2.639057
50%          4.077537
75%          6.202536
max         11.512925
Name: recency, dtype: float64

In [24]:
df["request_hour"] = df["request_time"].dt.hour
df["request_month"] = df["request_time"].dt.month
df["request_day"] = df["request_time"].dt.day
df["request_dayofweek"] = df["request_time"].dt.dayofweek

In [25]:
 df = df.sort_values(by='request_time')

In [26]:
num_data=len(df)

In [27]:
train = df.iloc[0:int(num_data*0.8),:]
test = df.iloc[int(num_data*0.8):len(df),:]

In [28]:
print(str(len(train)))
print(str(len(test)))

160000
40000


In [29]:
import datetime as dt
df_rt["request_time"] = pd.to_datetime(df_rt["request_time"])
df_rt_train = df_rt[df_rt["request_time"]<dt.datetime(2019,7,5,20,33,0)]

In [30]:
h = df_rt_train.groupby(["ssp_uid","sponsor_id"])["rt_status"].sum()
h=h.reset_index()
h["rt_times"] = h["rt_status"]
h=h.drop("rt_status",axis=1)
df = df.merge(h, on=["ssp_uid", "sponsor_id"], how="left")
df = df.fillna(0)

In [31]:
train_sub = df[df["request_time"]<dt.datetime(2019,7,5,20,33,0)]
unique_ad = train_sub.groupby(["sponsor_id","campaign_id","creative_id"])
unique_contents = train_sub.groupby(["sponsor_id","campaign_id"])
region_contents = train_sub.groupby(["sponsor_id","campaign_id","region"])

In [32]:
click_rate_by_contents = unique_ad["click_flg"].mean().reset_index()
click_rate_by_contents = click_rate_by_contents.rename(columns = {"click_flg":"click_flg_0"})
cv_by_contents = unique_ad["cv_flg"].mean().reset_index()
cv_by_contents = cv_by_contents.rename(columns = {"cv_flg":"cv_flg_0"})
click_rate_by_contents_1 = unique_contents["click_flg"].mean().reset_index()
click_rate_by_contents_1 = click_rate_by_contents_1.rename(columns = {"click_flg":"click_flg_1"})
cv_by_contents_1 = unique_contents["cv_flg"].mean().reset_index()
cv_by_contents_1 = cv_by_contents_1.rename(columns = {"cv_flg":"cv_flg_1"})
click_rate_by_contents_2 = region_contents["click_flg"].mean().reset_index()
click_rate_by_contents_2 = click_rate_by_contents_2.rename(columns = {"click_flg":"click_flg_2"})
cv_by_contents_2 = region_contents["cv_flg"].mean().reset_index()
cv_by_contents_2 = cv_by_contents_2.rename(columns = {"cv_flg":"cv_flg_2"})

In [33]:
df_final = df.merge(click_rate_by_contents, on=["sponsor_id","campaign_id","creative_id"], how="left")
df_final = df_final.merge(cv_by_contents, on=["sponsor_id","campaign_id","creative_id"], how="left")
df_final = df_final.merge(click_rate_by_contents_1, on=["sponsor_id","campaign_id"], how="left")
df_final = df_final.merge(cv_by_contents_1, on=["sponsor_id","campaign_id"], how="left")
df_final = df_final.merge(click_rate_by_contents_2, on=["sponsor_id","campaign_id","region"], how="left")
df_final = df_final.merge(cv_by_contents_2, on=["sponsor_id","campaign_id","region"], how="left")

In [34]:
df_final.to_csv("df_final.csv")

In [36]:
df_final.describe()

,ssp_uid,carrier_name,slot_id,sponsor_id,campaign_id,creative_type,creative_id,region,fq,inview_fq,...,request_month,request_day,request_dayofweek,rt_times,click_flg_0,cv_flg_0,click_flg_1,cv_flg_1,click_flg_2,cv_flg_2
count,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,199931.000000,199931.000000,199931.000000,199931.000000,199884.000000,199884.000000
mean,-2.828182e+15,-3.151998e+18,-2.979194e+17,-5.312828e+18,2.639002e+17,2.650965e+18,3.597165e+17,6.054026e+17,3.222790,0.121045,...,6.485390,16.392250,3.146965,0.150745,0.100488,0.000643,0.099871,0.000648,0.099981,0.000648
std,5.322411e+18,6.857739e+18,5.466563e+18,4.373663e+18,5.282961e+18,1.111413e+17,4.998125e+18,4.707350e+18,19.800552,1.836562,...,0.499788,11.762945,2.008431,0.714733,0.066818,0.001372,0.052912,0.000867,0.061538,0.002518
min,-9.223266e+18,-9.189916e+18,-9.223307e+18,-8.604110e+18,-8.847643e+18,4.528528e+17,-9.203191e+18,-9.142586e+18,0.000000,0.000000,...,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.005464,0.000000,0.000000,0.000000
25%,-4.613428e+18,-9.189916e+18,-5.357509e+18,-8.361557e+18,-2.138496e+18,2.656585e+18,-3.688929e+18,-1.000000e+00,0.000000,0.000000,...,6.000000,5.000000,1.000000,0.000000,0.053280,0.000000,0.058662,0.000144,0.055631,0.000000
50%,1.026416e+16,-9.189916e+18,-1.064948e+16,-8.361557e+18,1.683171e+18,2.656585e+18,1.367880e+18,-1.000000e+00,0.000000,0.000000,...,6.000000,25.000000,3.000000,0.000000,0.071429,0.000202,0.078351,0.000417,0.082342,0.000000
75%,4.602870e+18,4.940667e+18,4.016189e+18,1.480525e+18,3.028780e+18,2.656585e+18,3.772499e+18,5.951656e+18,1.000000,0.000000,...,7.000000,28.000000,5.000000,0.000000,0.148725,0.000650,0.139226,0.000647,0.137315,0.000482
max,9.223101e+18,4.940667e+18,9.216591e+18,1.592276e+18,8.865851e+18,2.656585e+18,9.215724e+18,8.943928e+18,3131.000000,202.000000,...,7.000000,30.000000,6.000000,128.000000,0.535714,0.015152,0.248691,0.005236,1.000000,0.071429


In [ ]:
df_final = pd.read_csv("df_final.csv")